In [80]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as PineconeLang
from pinecone import Pinecone as PineconeClient 
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [75]:
import os


In [76]:
PINECONE_API_KEY = ""
PINECONE_API_ENV = ""

In [77]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["PINECONE_API_ENV"] = PINECONE_API_ENV

In [3]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [10]:
#extracted_data = load_pdf("data/")
extracted_data = load_pdf(r"C:\Users\smsub\Subahani\Study\Generative AI\chatbotLlama2\GenAI-End-to-End-Medical-chatbot-using-LLAMA2\data")

In [12]:
extracted_data[0]

Document(page_content='', metadata={'source': 'C:\\Users\\smsub\\Subahani\\Study\\Generative AI\\chatbotLlama2\\GenAI-End-to-End-Medical-chatbot-using-LLAMA2\\data\\Medical_book.pdf', 'page': 0})

In [13]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [14]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [16]:
text_chunks[0].page_content

'TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION'

In [17]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [20]:
embeddings = download_hugging_face_embeddings()

In [21]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [22]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [23]:
query_result

[-0.034477222710847855,
 0.031023189425468445,
 0.00673493230715394,
 0.02610897831618786,
 -0.03936203941702843,
 -0.16030248999595642,
 0.06692398339509964,
 -0.006441492587327957,
 -0.04745054617524147,
 0.01475894171744585,
 0.07087540626525879,
 0.05552757903933525,
 0.019193286076188087,
 -0.026251375675201416,
 -0.010109512135386467,
 -0.026940539479255676,
 0.022307513281702995,
 -0.022226620465517044,
 -0.14969266951084137,
 -0.017493048682808876,
 0.00767626753076911,
 0.0543522946536541,
 0.003254437353461981,
 0.03172598034143448,
 -0.0846213847398758,
 -0.029406018555164337,
 0.05159567669034004,
 0.048124030232429504,
 -0.0033148014917969704,
 -0.05827920511364937,
 0.04196932911872864,
 0.022210698574781418,
 0.12818889319896698,
 -0.022338908165693283,
 -0.011656295508146286,
 0.06292832642793655,
 -0.03287626430392265,
 -0.09122607111930847,
 -0.03117542713880539,
 0.052699606865644455,
 0.0470348484814167,
 -0.08420299738645554,
 -0.030056146904826164,
 -0.02074474096

In [81]:
#Initializing the Pinecone
pc = PineconeClient(api_key=os.environ["PINECONE_API_KEY"],
                    environment=os.environ["PINECONE_API_ENV"])

index_name="myindex"

#Creating Embeddings for Each of The Text Chunks & storing
docsearch=PineconeLang.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

In [82]:
docsearch

In [83]:
#If we already have an index we can load it like this
docsearch=PineconeLang.from_existing_index(index_name, embeddings)

In [87]:
query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

docs[0].page_content

"GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"

In [88]:
print("Result", docs[0].page_content)

Result GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies
Allergic rhinitis is commonly triggered by
exposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.
The presence of an allergen causes the
body's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.
IgE molecules attach to mast
cells, which contain histamine.HistaminePollen grains
Lymphocyte
FIRST EXPOSURE


In [89]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [90]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [101]:
os.environ["HF_AUTH_TOKEN"]  = ""

In [103]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q8_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [98]:
#from ctransformers import AutoModelForCausalLM

#llm = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-7B-Chat-GGML", model_file="llama-2-7b-chat.ggmlv3.q8_0.bin")


Fetching 1 files: 100%|██████████| 1/1 [12:59<00:00, 779.03s/it]


In [100]:
#from huggingface_hub import hf_hub_download

## https://huggingface.co/docs/huggingface_hub/v0.16.3/en/package_reference/file_download#huggingface_hub.hf_hub_download
#hf_hub_download(
    #repo_id="TheBloke/Llama-2-7B-Chat-GGML",
    #filename="llama-2-7b-chat.ggmlv3.q8_0.bin",
    #local_dir=r"C:\Users\smsub\Subahani\Study\Generative AI\chatbotLlama2\GenAI-End-to-End-Medical-chatbot-using-LLAMA2\model"
#)

'C:\\Users\\smsub\\Subahani\\Study\\Generative AI\\chatbotLlama2\\GenAI-End-to-End-Medical-chatbot-using-LLAMA2\\model\\llama-2-7b-chat.ggmlv3.q8_0.bin'

In [104]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [105]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

c:\Users\smsub\Anaconda3\envs\mchatbot\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response :  Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.
Response :  Psychotherapy is a type of treatment that helps people manage their mental health issues by talking with a trained mental health professional.
Response :  GBS (Group B streptococci) is a type of bacteria that, if passed to a baby during birth, can cause infection in the baby's lungs, bloodstream, or brain. GBS is a common cause of infections in newborns and can be dangerous, especially for premature babies.
